# __Convolutional Neural Network to identify fruits__


<b><h2> Import required packages</h2><b>

In [1]:
#import all the necessary packages and functions
import numpy as np
import tensorflow as tf
import cv2
import os
from sklearn.preprocessing import LabelBinarizer
from sklearn.utils import shuffle
from keras.models import Sequential, save_model
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten, BatchNormalization

Using TensorFlow backend.


<b><h2>Load the data</h2></b>

In [2]:
def read_data(folder_name):
    """
    This function traverses the given directory, opens each sub-folder and reads all the image files one by one into a list. 
    This list is converted into a numpy array.
   
    Parameters:
    folder_name - Relative path of directory that needs to be traversed
    
    Returns:
    x - A numpy array of size (num_images, 100, 100, 3) where num_images is total number of images read
    """
    x=[]
    for folder in os.listdir(folder_name):
        for file in os.listdir(os.path.join(folder_name,folder)):
            x.append(cv2.imread(os.path.join(folder_name,folder,file)))
    x = np.array(x)
    return x

In [3]:
def read_labels(folder_name):
    """
    This function traverses the given directory. Each sub-folder is opened and for each file inside it, the name of the sub-folder
    is added to the label vector
    
    Parameters:
    folder_name - Relative path of directory that needs to be traversed
    
    Returns:
    y - A vector containing labels corresponding to each image in the sub-folders
    """
    y=[]
    for folder in os.listdir(folder_name):
        for file in os.listdir(os.path.join(folder_name,folder)):
            y.append(str(folder))
    return y

In [4]:
## Train and test data directories
train_folder = 'data/fruits-360/Training'
test_folder = 'data/fruits-360/Test'

## Create numpy arrays of train and test images
train_data = read_data(train_folder)
test_data = read_data(test_folder)

## Create label array for train and test images
train_labels = read_labels(train_folder)
test_labels = read_labels(test_folder)

<b><h2>One-hot encoding of labels</h2></b>

In [5]:
## Create one-hot vectors of labels using LabelBinarizer class of sklearn.  
lb = LabelBinarizer()
## Fit the LabelBinarizer to training labels
lb.fit(train_labels)
# Transform the label vectors
y_train = lb.transform(train_labels)
y_test = lb.transform(test_labels)

<b><h2>Shuffling training data</h2></b>

In [6]:
# Shuffle the dataset using shuffle function of sklearn
X,Y=shuffle(train_data, y_train, random_state=0)

<b><h2>Splitting into validation and test set after shuffling</h2></b>

In [9]:
X_1, Y_1 = shuffle(test_data, y_test, random_state=0)
X_val = X_1[:7000,:,:,:]
Y_val = Y_1[:7000,]
X_test = X_1[7000:,:,:,:]
Y_test = Y_1[7000:,]

<b><h2>Defining the model</h2></b>

In [11]:
def createModel():
    '''
    Create a convolutional neural network model using Keras.
    '''
    model = Sequential()
    model.add(BatchNormalization(axis=-1, input_shape=[100,100,3]))
    model.add(Conv2D(64, (5, 5), padding='same', activation='relu', data_format='channels_last'))
    model.add(Conv2D(64, (5, 5), padding='same', activation='relu', data_format='channels_last'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
    model.add(BatchNormalization(axis=-1))
 
    model.add(Conv2D(128, (5, 5), padding='same', activation='relu', data_format='channels_last'))
    model.add(Conv2D(128, (5, 5), padding='same', activation='relu', data_format='channels_last'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
    model.add(BatchNormalization(axis=-1))
    
    model.add(Conv2D(128, (5, 5), padding='same', activation='relu', data_format='channels_last'))
    model.add(Conv2D(128, (5, 5), padding='same', activation='relu', data_format='channels_last'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
    model.add(BatchNormalization(axis=-1))
 
    model.add(Conv2D(256, (5, 5), padding='same', activation='relu', data_format='channels_last'))
    model.add(Conv2D(256, (5, 5), padding='same', activation='relu', data_format='channels_last'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
    model.add(BatchNormalization(axis=-1))
 
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(81, activation='softmax'))
     
    return model

In [12]:
## Create model instance
model1 = createModel()
## Compile the model
model1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

<b><h2>Model summary</h2><b>

In [13]:
model1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_1 (Batch (None, 100, 100, 3)       12        
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 100, 100, 64)      4864      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 100, 100, 64)      102464    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 50, 50, 64)        0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 50, 50, 64)        256       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 50, 50, 128)       204928    
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 50, 50, 128)       409728    
__________

<b><h2>Training the model using GPU</h2></b>

In [14]:
## Use gpu to train the model and perform validation on test data
with tf.device('/gpu:0'):
    model1.fit(X, Y, batch_size=32, epochs=2, verbose=1, validation_data=(X_val, Y_val))

Train on 41322 samples, validate on 7000 samples
Epoch 1/2
41322/41322 [==============================] - 1553s 38ms/step - loss: 0.5292 - acc: 0.8544 - val_loss: 3.1076 - val_acc: 0.5316
Epoch 2/2
41322/41322 [==============================] - 1542s 37ms/step - loss: 0.1083 - acc: 0.9674 - val_loss: 0.3143 - val_acc: 0.9190


<br><br>The model achieved 96% training set accuracy and 91% validation set accuracy in just 2 epochs. Training for few more epochs would definitely help improve the accuracy. Let's see how well it performs on the test set.<br>

<b><h2>Evaluating the model on test set</h2></b>

In [20]:
model1.evaluate(X_test, Y_test, verbose=1)

6877/6877 [==============================] - 83s 12ms/step


[0.32147629967809016, 0.9126072414690661]

<br><br>The model achieved an accuracy of 91% on the test set. It's not anything to be excited about, but I'm positive that the model can perform better if trained for more epochs.<br><br>

In [19]:
# Save model to a HDF5 file
save_model(model1, 'fruit_model')